# **Homework 5: Basic Block Recursivity**

## **ECON8582: Macro-Labour**

#### *Conor Bayliss*

In [1]:
using LinearAlgebra, QuantEcon, Random, CairoMakie

#### **Calibration**

In [21]:
pars = (;β = 0.996, # discount factor
        ρ = 0.98, # autocorrelation of productivity process
        σe = 0.01, # standard deviation of productivity process
        nz = 15, # number of points in the productivity grid
        γ = 0.6, # coefficient of job finding rate
        κ = 1.89, # vacancy posting cost
        δ = 0.012, # separation rate
        σ = 0, # CRRA parameter
        α = 1, # search cost parameter
        χ = 2, # search cost exponent
        a = 1/3, # λ(s) = s^a job posting opportunities
        w_lb = .1, # lower bound of wage grid
        w_ub = 1, # upper bound of wage grid
        Δw = 0.025, # wage grid step
        b = 0.54, # benefit parameter
        s_lb = 0.2, # lower bound of search cost grid
        s_ub = 0.4, # upper bound of search cost grid
        Δs = 0.02, # search cost grid step
        toler = 1e-6, # tolerance for convergence
        maxiter = 100000) # maximum number of iterations

(β = 0.996, ρ = 0.98, σe = 0.01, nz = 15, γ = 0.6, κ = 1.89, δ = 0.012, σ = 0, α = 1, χ = 2, a = 0.3333333333333333, w_lb = 0.1, w_ub = 1, Δw = 0.025, b = 0.54, s_lb = 0.2, s_ub = 0.4, Δs = 0.02, toler = 1.0e-6, maxiter = 100000)

##### **Income process**

In [3]:
function income_process(p)
    (;ρ, σe, nz) = p
    mc = rouwenhorst(nz, ρ, σe)
    z_grid = exp.(mc.state_values)
    Π = mc.p
    return z_grid, Π 
end

test_z_grid, test_Π = income_process(pars)

([0.8285936952614785, 0.8511519660876842, 0.8743243805956225, 0.8981276587043299, 0.9225789755287533, 0.9476959737723538, 0.9734967764570943, 1.0, 1.027224767645724, 1.0551907232648117, 1.0839180455276196, 1.1134274624641165, 1.14374026642007, 1.174878329420415, 1.2068641189508822], [0.8687458127689781 0.1228529432198556 … 1.386000000000016e-25 1.0000000000000126e-28; 0.008775210229989686 0.8698981131022091 … 1.2741400000000137e-23 9.900000000000117e-27; … ; 9.900000000000117e-27 1.2741400000000137e-23 … 0.8698981131022091 0.008775210229989686; 1.0000000000000126e-28 1.386000000000016e-25 … 0.1228529432198556 0.8687458127689781])

##### **Wage grid**

In [4]:
function wage_grid(pars)
    (;w_lb, w_ub, Δw) = pars
    w_grid = range(w_lb, w_ub, step = Δw)
    return w_grid
end

test_w_grid = wage_grid(pars)

0.1:0.025:1.0

##### **Search grid**

In [5]:
function search_cost_grid(pars)
    (;s_lb, s_ub, Δs) = pars
    s_grid = range(s_lb, s_ub, step = Δs)
    return s_grid
end

test_s_grid = search_cost_grid(pars)

0.2:0.02:0.4

##### **Job finding rate**

In [6]:
function job_finding_rate(θ, pars)
    (;γ) = pars
    p = θ*(1. + θ^γ)^(-1/γ)
    return p
end

test_find_rate = job_finding_rate(test_s_grid[1], pars)

0.11681940254944001

##### **Job posting opportunities**

In [7]:
function job_opp(s, pars)
    (;a) = pars
    job_opp = s^a
    return job_opp
end

test_job_opp = job_opp(test_s_grid[1],pars)

0.5848035476425733

##### **Benefits grid**

In [8]:
function benefits_grid(pars)
    (;b) = pars
    temp_w_grid = wage_grid(pars)
    benefits = b .* temp_w_grid
    return benefits
end

test_benefits = benefits_grid(pars)

0.054000000000000006:0.013500000000000002:0.54

##### **Utility**

In [9]:
function utility(c,pars)
    (;σ) = pars
    
    if σ == 1
        u = log
    else
        u = x -> (x^(1-σ) - 1)/(1-σ)
    end

    return u(c)
end

test_c = utility(2,pars)

1.0

##### **Job destruction**

In [10]:
function job_dest(z,w,pars)
    (;δ) = pars
    if z >= w
        return δ
    else
        return 1
    end
end

test_job_dest = job_dest(test_z_grid[5],test_w_grid[34],pars)

1

##### **Search costs**

In [11]:
function search_costs(s,pars)
    (;α, χ) = pars
    return α * (s^χ)
end

test_search_costs = search_costs(test_s_grid[1],pars)

0.04000000000000001

##### **Initial guess for $\mathbf{J}$**

In [12]:
function initial_J(pars)
    (;nz,w_ub,w_lb,Δw) = pars
    nw = (w_ub - w_lb)/Δw + 1
    nw = round(Int,nw)
    J = zeros(nw,nz)
    return J
end

test_J = initial_J(pars)

37×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0 

#### **Value function iteration on $\mathbf{J}$**

In [17]:
function iterate_J(pars)
    (;nz, w_ub, w_lb, Δw, a, b, γ, κ, δ, α, χ, β, toler, maxiter) = pars
    nw = (w_ub - w_lb)/Δw + 1
    nw = round(Int,nw)
    J_init = initial_J(pars)
    J_new = copy(J_init)
    error = 1
    iter = 1
    z_grid, Π = income_process(pars)
    w_grid = wage_grid(pars)
    EV = zeros(nw,nz)
    while iter <= maxiter
        J_new = copy(J_init)
        for i in 1:nw
            for j in 1:nz
                EV[i,j] = 0
                for jp in 1:nz
                    EV[i,j] += J_init[i,jp] * Π[j,jp]
                end
            end
            J_init[i,:] = z_grid .- w_grid[i] .+ (β .* (1 .- job_dest.(z_grid,w_grid[i],Ref(pars)))) .* EV[i,:]
        end
        error = maximum(abs.(J_new - J_init))
        J_new = copy(J_init)
        if error < toler
            println("--------------------")
            println("Converged in $iter iterations")
            println("Error: $error")
            println("--------------------")
            break
        end
        if iter == maxiter
            println("Maximum number of iterations reached")
        end
        if iter % 100 == 0
            println("--------------------")
            println("Iteration: $iter")
            println("Error: $error")
            println("--------------------")
        end
        iter += 1
    end
    return J_new
end

 @time J_out = iterate_J(pars)

--------------------
Iteration: 100
Error: 0.18867370091780344
--------------------
--------------------
Iteration: 200
Error: 0.03687394205703498
--------------------
--------------------
Iteration: 300
Error: 0.007360990328706407
--------------------
--------------------
Iteration: 400
Error: 0.0014735920943351744
--------------------
--------------------
Iteration: 500
Error: 0.0002951078404294094
--------------------
--------------------
Iteration: 600
Error: 5.9102486900997064e-5
--------------------
--------------------
Iteration: 700
Error: 1.1836781190766033e-5
--------------------
--------------------
Iteration: 800
Error: 2.3706195619865866e-6
--------------------
--------------------
Converged in 854 iterations
Error: 9.948122894343214e-7
--------------------
  0.252056 seconds (243.96 k allocations: 40.930 MiB, 18.70% gc time, 72.04% compilation time)


37×15 Matrix{Float64}:
 51.5029     52.1771    52.8629       …  60.4167   61.2475   62.093
 49.9357     50.6099    51.2957          58.8495   59.6803   60.5258
 48.3685     49.0427    49.7285          57.2823   58.1131   58.9586
 46.8013     47.4755    48.1613          55.7151   56.5459   57.3914
 45.2341     45.9083    46.5941          54.1479   54.9787   55.8242
 43.6669     44.3411    45.0269       …  52.5807   53.4115   54.257
 42.0997     42.7739    43.4597          51.0135   51.8443   52.6898
 40.5325     41.2067    41.8925          49.4463   50.2771   51.1226
 38.9653     39.6395    40.3253          47.8791   48.7099   49.5554
 37.3981     38.0723    38.7581          46.3119   47.1427   47.9882
  ⋮                                   ⋱                      
  7.62132     8.29549    8.98132         16.5351   17.3659   18.2114
  6.05412     6.72829    7.41412         14.9679   15.7987   16.6442
 -0.0214063   4.85121    5.7971       …  13.4004   14.2312   15.0768
 -0.0464063  -0.0238

#### **Solve for $\theta$, setting $\theta = 0$ in cases where $\mathbf{J}<0$.** 

To do so, first notice that we have
$$
\theta(w;\psi) = \frac{v(w;\psi)}{u(w;\psi)}, \quad q(\theta(w;\psi)) = \frac{M(w;\psi)}{v(w;\psi)}, \quad p(\theta(w;\psi)) = \frac{M(w;\psi)}{u(w;\psi)}
$$
and rearranging the last two conditions gives
$$
v(w;\psi) = \frac{M(w;\psi)}{q(\theta(w;\psi))}, \quad u(w;\psi) = \frac{M(w;\psi)}{p(\theta(w;\psi))}
$$
and, plugging into his first condition, we obtain
$$
p(\theta(w;\psi)) = \theta(w;\psi) q(\theta(w;\psi)).
$$
In the problem, we are told to assume that 
$$
p(\theta(w;\psi)) = \theta(w;\psi) (1+\theta(w;\psi)^\gamma)^{-\frac{1}{\gamma}}
$$
and therefore we have
$$
q(\theta(w;\psi)) = (1+\theta(w;\psi)^\gamma)^{-\frac{1}{\gamma}}.
$$
Since we want to find the $\theta$'s themselves, we then use the free-entry condition
$$
\kappa = q(\theta(w;\psi))\mathbf{J}(w;\psi)
$$
$$
\implies \kappa = (1+\theta(w;\psi)^\gamma)^{-\frac{1}{\gamma}} \mathbf{J}(w;\psi)
$$
by our functional form for $q$. Then, rearranging, we find
$$
\theta(w;\psi) = \left[\left(\frac{\mathbf{J}(w;\psi)}{\kappa}\right)^\gamma  - 1\right]^{\frac{1}{\gamma}}.
$$

In [18]:
function thetas(J,pars)
    (;γ, κ, nz, w_lb,w_ub,Δw) = pars
    nw = (w_ub - w_lb)/Δw + 1
    nw = round(Int,nw)
    Θ = zeros(nw,nz)
    for i in 1:nw
        for j in 1:nz
            if J[i,j] / κ >= 1.0
                Θ[i,j] = ((J[i,j] / κ)^γ - 1.)^(1/γ)
            else
                Θ[i,j] = 0
            end
        end
    end
    return Θ
end

test_thetas = thetas(J_out,pars)

37×15 Matrix{Float64}:
 21.29     21.6133    21.9425    …  25.5845    25.9867    26.3965
 20.5395   20.8621    21.1907       24.8266    25.2282    25.6373
 19.7905   20.1125    20.4404       24.0698    24.4708    24.8793
 19.0432   19.3644    19.6916       23.3142    23.7145    24.1224
 18.2975   18.6181    18.9445       22.5599    22.9596    23.3667
 17.5537   17.8734    18.1991    …  21.807     22.2059    22.6124
 16.8117   17.1307    17.4555       21.0555    21.4537    21.8593
 16.0718   16.3899    16.7138       20.3054    20.7028    21.1077
 15.3341   15.6512    15.9742       19.557     19.9535    20.3576
 14.5987   14.9147    15.2367       18.8101    19.2058    19.609
  ⋮                              ⋱                        
  1.56551   1.81296    2.07055       5.1523     5.50924    5.87508
  1.01795   1.24812    1.49074       4.48638    4.83808    5.19901
  0.0       0.633606   0.932713  …   3.83162    4.1771     4.53227
  0.0       0.0        0.0           3.18025    3.51925   

#### **Initial guess for $\mathbf{W}$**

In [19]:
function initial_W(pars)
    (;nz,w_ub,w_lb,Δw) = pars
    nw = (w_ub - w_lb)/Δw + 1
    nw = round(Int,nw)
    W = zeros(nw,nz)
    return W
end

test_W = initial_W(pars)

37×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0 

#### **Initial guess for $\mathbf{U}$**

In [20]:
function initial_U(pars)
    (;nz,w_ub,w_lb,Δw) = pars
    nw = (w_ub - w_lb)/Δw + 1
    nw = round(Int,nw)
    U = zeros(nw,nz)
    return U
end

test_U = initial_U(pars)

37×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0 

#### **Value function iteration to find $\mathbf{W}$ and $\mathbf{U}$**

In [24]:
function iterate_W_U(pars)
    (;nz, w_ub, w_lb, Δw, s_lb, s_ub, Δs, a, b, γ, κ, δ, α, χ, β, toler, maxiter) = pars
    nw = (w_ub - w_lb)/Δw + 1
    nw = round(Int,nw)
    ns = (s_ub - s_lb)/Δs + 1
    ns = round(Int,ns)
    W_init = initial_W(pars)
    U_init = initial_U(pars)
    W_new = copy(W_init)
    U_new = copy(U_init)
    EW = zeros(nw,nz)
    EU = zeros(nw,nz,ns)
    final_util = zeros(ns)
    z_grid, Π = income_process(pars)
    w_grid = wage_grid(pars)
    b_grid = benefits_grid(pars)
    s_grid = search_cost_grid(pars)
    Θ = thetas(J_out,pars)
    error = toler+1
    iter=0
    while iter <= maxiter
        for i in 1:nw
            for j in 1:nz
                EW[i,j] = 0
                for jp in 1:nz
                    EW[i,j] += Π[j,jp] * (1-job_dest(z_grid[jp],w_grid[i],pars)) * U_init[i,jp] + Π[j,jp] * job_dest(z_grid[jp],w_grid[i],pars) * W_init[i,jp]
                end
                W_new[i,j] = utility(w_grid[i],pars) + β * EW[i,j]
            end
        end
        for i in 1:nw
            for j in 1:nz
                for s in 1:ns
                    EU[i,j,s] = 0
                    util = utility(b_grid[i],pars) - search_costs(s_grid[s],pars) 
                    λ = job_opp(s_grid[s],pars)
                    search_max = 0
                    for jp in 1:nz
                        on_job_search = job_opp(Θ[i,jp],pars) .* W_new[i,jp] .+ (1 - job_opp(Θ[i,jp],pars)) .* (0.1 * U_new[1,jp] .+ 0.9 * U_new[i,jp])
                        if on_job_search > search_max
                            search_max = on_job_search
                        end
                    end
                    for jp in 1:nz
                        EU[i,j,s] += Π[j,jp] * λ * search_max + Π[j,jp] * (1 - λ) * U_new[i,jp]
                    end
                    final_util[s] = util .+ β * EU[i,j,s]
                end
                U_new[i,j] = maximum(final_util)
            end
        end                            
        errorW = maximum(abs.(W_new - W_init))
        errorU = maximum(abs.(U_new - U_init))
        error = max(errorW,errorU)
        if error < toler
            println("--------------------")
            println("Converged in $iter iterations")
            println("Error: $error")
            println("--------------------")
            break
        end
        if iter == maxiter
            println("Maximum number of iterations reached")
        end
        if iter % 100 == 0
            println("--------------------")
            println("Iteration: $iter")
            println("Error: $error")
            println("--------------------")
        end
        W_init = copy(W_new)
        U_init = copy(U_new)
        iter += 1
    end
    return W_init,U_init
end

@time W_out, U_out = iterate_W_U(pars)

--------------------
Iteration: 0
Error: 1.0423160771187323
--------------------
--------------------
Iteration: 100
Error: 0.01045854612986119
--------------------
--------------------
Iteration: 200
Error: 0.001021224470738602
--------------------
--------------------
Iteration: 300
Error: 9.359937185537959e-5
--------------------
--------------------
Iteration: 400
Error: 8.569029967375741e-6
--------------------
--------------------
Converged in 490 iterations
Error: 9.963608005314484e-7
--------------------
  2.520730 seconds (92.82 k allocations: 18.684 MiB, 16.51% compilation time)


([-2.4037498840962956 -2.4037498840962956 … -2.4037498840962956 -2.4037498840962956; -2.3602253629398673 -2.3602253629398673 … -2.3602253629398673 -2.3602253629398673; … ; -2.5208151121131066 -2.4151316490581234 … -0.8619991989637525 -0.8619991989425935; -0.565204181805202 -0.5812257294090186 … -0.8166516973638313 -0.8166516973670385], [-1.4989312162430868 -1.4989312162430868 … -1.4989312162430868 -1.4989312162430868; -1.480635039552959 -1.480635039552959 … -1.480635039552959 -1.480635039552959; … ; -0.8400978250209097 -0.8400978250209095 … -0.8400978250209096 -0.8400978250209097; -0.8199712577843838 -0.8199712577843837 … -0.8199712577843836 -0.8199712577843836])

In [25]:
@show W_out

W_out = [-2.4037498840962956 -2.4037498840962956 -2.403749884096296 -2.4037498840962956 -2.4037498840962956 -2.403749884096296 -2.403749884096296 -2.4037498840962956 -2.4037498840962956 -2.403749884096296 -2.403749884096296 -2.4037498840962956 -2.403749884096296 -2.4037498840962956 -2.4037498840962956; -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673 -2.3602253629398673; -2.3167008417834385 -2.3167008417834385 -2.3167008417834385 -2.316700841783439 -2.3167008417834385 -2.3167008417834385 -2.316700841783439 -2.316700841783439 -2.3167008417834385 -2.3167008417834385 -2.316700841783439 -2.3167008417834385 -2.3167008417834385 -2.316700841783439 -2.316700841783439; -2.2731763206270097 -2.2731763206270097 -2.2731763206270097 -2.2731763206270097 -2.2731763206270097 

37×15 Matrix{Float64}:
 -2.40375   -2.40375   -2.40375   …  -2.40375   -2.40375   -2.40375
 -2.36023   -2.36023   -2.36023      -2.36023   -2.36023   -2.36023
 -2.3167    -2.3167    -2.3167       -2.3167    -2.3167    -2.3167
 -2.27318   -2.27318   -2.27318      -2.27318   -2.27318   -2.27318
 -2.22965   -2.22965   -2.22965      -2.22965   -2.22965   -2.22965
 -2.18613   -2.18613   -2.18613   …  -2.18613   -2.18613   -2.18613
 -2.1426    -2.1426    -2.1426       -2.1426    -2.1426    -2.1426
 -2.09908   -2.09908   -2.09908      -2.09908   -2.09908   -2.09908
 -2.05555   -2.05555   -2.05555      -2.05555   -2.05555   -2.05555
 -2.01203   -2.01203   -2.01203      -2.01203   -2.01203   -2.01203
  ⋮                               ⋱                        
 -1.1762    -1.1762    -1.1762       -1.1762    -1.1762    -1.1762
 -1.13167   -1.13167   -1.13167      -1.13167   -1.13167   -1.13167
 -2.02622   -1.09652   -1.08703   …  -1.08693   -1.08693   -1.08693
 -3.81378   -3.0362    -2.1168      

In [26]:
@show U_out

U_out = [-1.4989312162430868 -1.4989312162430868 -1.4989312162430868 -1.4989312162430868 -1.4989312162430868 -1.4989312162430868 -1.498931216243087 -1.4989312162430868 -1.4989312162430868 -1.4989312162430868 -1.498931216243087 -1.4989312162430868 -1.498931216243087 -1.4989312162430868 -1.4989312162430868; -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.4806350395529588 -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.480635039552959 -1.480635039552959; -1.462338862862831 -1.462338862862831 -1.4623388628628309 -1.462338862862831 -1.462338862862831 -1.462338862862831 -1.462338862862831 -1.462338862862831 -1.462338862862831 -1.462338862862831 -1.4623388628628313 -1.462338862862831 -1.462338862862831 -1.462338862862831 -1.4623388628628313; -1.4440426861727031 -1.4440426861727031 -1.444042686172703 -1.444042686172703 -1.444042686172703 -1.444042686172703 -

37×15 Matrix{Float64}:
 -1.49893   -1.49893   -1.49893   …  -1.49893   -1.49893   -1.49893
 -1.48064   -1.48064   -1.48064      -1.48064   -1.48064   -1.48064
 -1.46234   -1.46234   -1.46234      -1.46234   -1.46234   -1.46234
 -1.44404   -1.44404   -1.44404      -1.44404   -1.44404   -1.44404
 -1.42575   -1.42575   -1.42575      -1.42575   -1.42575   -1.42575
 -1.40745   -1.40745   -1.40745   …  -1.40745   -1.40745   -1.40745
 -1.38915   -1.38915   -1.38915      -1.38915   -1.38915   -1.38915
 -1.37086   -1.37086   -1.37086      -1.37086   -1.37086   -1.37086
 -1.35256   -1.35256   -1.35256      -1.35256   -1.35256   -1.35256
 -1.33427   -1.33427   -1.33427      -1.33427   -1.33427   -1.33427
  ⋮                               ⋱                        
 -0.977743  -0.977743  -0.977743     -0.977743  -0.977743  -0.977743
 -0.958434  -0.958434  -0.958434     -0.958434  -0.958434  -0.958434
 -0.938921  -0.938921  -0.938921  …  -0.938921  -0.938921  -0.938921
 -0.919221  -0.919221  -0.9192